#This notebook is for preprocessing the raw parcels dataset into new formats that can be used to automatically search for the longitude and latitude of that parcel.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import json

#Load the raw parcels dataset.

In [ ]:
PARCELS = "/content/gdrive/My Drive/Colab Notebooks/data/raw_boston_parcels.csv"

#Take a look.

In [ ]:
df = pd.read_csv(PARCELS)
df.head(30)

,WKT,CITY,MAIL_ADDRESS,MAIL_CS,GIS_ID,PID_LONG,EGIS_WM_dbo_Live_Assessing_Data,PID,ZIPCODE,FULL_ADDRESS,STArea,ADDRESSEE,OWNER,MAIL_ZIPCODE,STLength
0,POLYGON Z ((-7906937.0588138 5218206.85065898 ...,EAST BOSTON,288 LEXINGTON ST,EAST BOSTON MA,100240000,100240000,NaN,100240000,2128.0,"288 LEXINGTON ST, 02128",240.296875,VIPOND ALEX RICHARD,VIPOND ALEX RICHARD,02128,79.257872
1,POLYGON Z ((-7906706.65043381 5218263.30970503...,EAST BOSTON,335 E EAGLE ST,EAST BOSTON MA,100262000,100262000,NaN,100262000,2128.0,"335 E EAGLE ST, 02128",279.589844,ALFARO MAREL W,ALFARO MAREL W,02128,79.238006
2,POLYGON Z ((-7906737.20932718 5217898.46786157...,EAST BOSTON,TWO JERICHO PZ WING C STE 11,JERICHO NY,100335000,100335000,NaN,100335000,2128.0,"BREMEN ST, 02128",104.550781,GAVEGNANO HENRY J,GAVEGNANO HENRY J,11753,42.711702
3,POLYGON Z ((-7906364.58110764 5218052.8044199 ...,EAST BOSTON,1 HARBORSIDE DR #200S,EAST BOSTON MA,100403000,100403000,NaN,100403000,2128.0,"5 VIENNA ST, 02128",418.890625,MASSACHUSETTS PORT AUTHORITY,MASSACHUSETTS PORT AUTHORITY,02128,102.510882
4,POLYGON Z ((-7906047.58426405 5218969.77038667...,EAST BOSTON,CHELSEA,EAST BOSTON MA,100439020,100439020,NaN,100439020,2128.0,"CHELSEA ST, 02128",235.000000,MASS WATER RESOURCE AUTHORTY,MASS WATER RESOURCE AUTHORTY,02128,114.954784
5,POLYGON Z ((-7906468.79864987 5218210.95959695...,EAST BOSTON,581 BOYLSTON ST STE 604,BOSTON MA,100442000,100442000,NaN,100442000,2128.0,"546 SARATOGA ST, 02128",322.167969,FIVE46 SARATOGA STREET LLC,FIVE46 SARATOGA STREET LLC,02116,82.927493
6,POLYGON Z ((-7905290.65847483 5218711.11070917...,EAST BOSTON,668 BENNINGTON ST,E BOSTON MA,101047000,101047000,NaN,101047000,2128.0,"668 BENNINGTON ST, 02128",243.078125,TETZAGUIC HILDA,TETZAGUIC HILDA,02128,65.903682
7,POLYGON Z ((-7905603.86597413 5218392.84915356...,EAST BOSTON,65 HORACE ST,EAST BOSTON MA,101150000,101150000,NaN,101150000,2128.0,"65 HORACE ST, 02128",449.277344,COUGHLIN JAMES G,COUGHLIN JAMES G,02128,103.643530
8,POLYGON Z ((-7904085.17320456 5218791.46180303...,EAST BOSTON,37 ST ANDREW RD,EAST BOSTON MA,101199000,101199000,NaN,101199000,2128.0,"37 39 ST ANDREW RD, 02128",966.390625,FERULLO JAMES A,FERULLO JAMES A,02128,133.776553
9,POLYGON Z ((-7904046.81009704 5218926.28398474...,EAST BOSTON,1084 SARATOGA,EAST BOSTON MA,101334000,101334000,NaN,101334000,2128.0,"1084 SARATOGA ST, 02128",420.816406,DA SILVA EDIVALDO BENTO,DA SILVA EDIVALDO BENTO,02128,84.581491


#Remove some unrelevent info, and check the shape.

In [ ]:
print(df.shape)
df = df.drop(columns=['WKT', 'GIS_ID', 'PID_LONG', 'EGIS_WM_dbo_Live_Assessing_Data',	'PID', 'STArea', 'STLength'])
df = df.dropna()
print(df.shape)
df.head()

(171922, 15)
(171904, 8)


,CITY,MAIL_ADDRESS,MAIL_CS,ZIPCODE,FULL_ADDRESS,ADDRESSEE,OWNER,MAIL_ZIPCODE
0,EAST BOSTON,288 LEXINGTON ST,EAST BOSTON MA,2128.0,"288 LEXINGTON ST, 02128",VIPOND ALEX RICHARD,VIPOND ALEX RICHARD,02128
1,EAST BOSTON,335 E EAGLE ST,EAST BOSTON MA,2128.0,"335 E EAGLE ST, 02128",ALFARO MAREL W,ALFARO MAREL W,02128
2,EAST BOSTON,TWO JERICHO PZ WING C STE 11,JERICHO NY,2128.0,"BREMEN ST, 02128",GAVEGNANO HENRY J,GAVEGNANO HENRY J,11753
3,EAST BOSTON,1 HARBORSIDE DR #200S,EAST BOSTON MA,2128.0,"5 VIENNA ST, 02128",MASSACHUSETTS PORT AUTHORITY,MASSACHUSETTS PORT AUTHORITY,02128
4,EAST BOSTON,CHELSEA,EAST BOSTON MA,2128.0,"CHELSEA ST, 02128",MASS WATER RESOURCE AUTHORTY,MASS WATER RESOURCE AUTHORTY,02128


#See how many cities are in the dataset. Some do not belong to Boston City. (e.g., Brookline)

In [ ]:
cities = set(df['CITY'].to_list())
print(cities)

{'ALLSTON', 'BRIGHTON', 'BROOKLINE', 'HYDE PARK', 'ROSLINDALE', 'READVILLE', 'MATTAPAN', 'CHARLESTOWN', 'EAST BOSTON', 'DORCHESTER', 'JAMAICA PLAIN', 'WEST ROXBURY', 'ROXBURY CROSSIN', 'CHESTNUT HILL', 'SOUTH BOSTON', 'BOSTON', 'JAMAIA PLAIN', 'ROXBURY'}


In [ ]:
def is_city_Boston(zipcode):
  """zipcode: an integer number which is the zipcode"""
  """return true if it is within city of Boston; false otherwise"""
  zip = [2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 
         2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2163,
         2196, 2199, 2201, 2203, 2204, 2205, 2206, 2210, 2211, 2212, 2215, 2217, 2222, 2241, 2266, 
         2283, 2284, 2293, 2297, 2298]
  try:
    if int(zipcode) in zip:
      return 1
    else:
      return 0
  except:
    return 0

#Remove those wrong parcels and check the shape again. 1000+ parcels are excluded.

In [ ]:
# should use zipcode and fulladdress instead of mail ones because the owners could live elsewhere! 
df = df[df['ZIPCODE'].map(lambda x: is_city_Boston(x)) == 1]
print(df.shape)

(170860, 8)


In [ ]:
cities = set(df['CITY'].to_list())
print(cities)

{'ALLSTON', 'BRIGHTON', 'HYDE PARK', 'ROSLINDALE', 'READVILLE', 'MATTAPAN', 'CHARLESTOWN', 'EAST BOSTON', 'DORCHESTER', 'JAMAICA PLAIN', 'WEST ROXBURY', 'ROXBURY CROSSIN', 'SOUTH BOSTON', 'BOSTON', 'JAMAIA PLAIN', 'ROXBURY'}


#Clean the dataset.

In [ ]:
# need to remove things like apt number; first need to inspect the data

def has_sign(x):
  return ('#' in x)

temp = df[df['FULL_ADDRESS'].map(lambda x: has_sign(x)) == True]
temp.head(30)

,CITY,MAIL_ADDRESS,MAIL_CS,ZIPCODE,FULL_ADDRESS,ADDRESSEE,OWNER,MAIL_ZIPCODE
18,EAST BOSTON,22 SWAN AVENUE,EAST BOSTON MA,2128.0,"12 LEVERETT AV #9-C, 02128",LEVERETT REALTY TRUST,LEVERETT REALTY TRUST,02128
19,EAST BOSTON,12 LEVERETT AV #10C,E BOSTON MA,2128.0,"12 LEVERETT AV #10-C, 02128",LYNCH KATHLEEN,LYNCH KATHLEEN,02128
20,EAST BOSTON,12 LEVERETT AV #11-C,EAST BOSTON MA,2128.0,"12 LEVERETT AV #11-C, 02128",DONES KEVEN,DONES KEVEN,02128
21,EAST BOSTON,12 LEVERETT AV #12-C,EAST BOSTON MA,2128.0,"12 LEVERETT AV #12-C, 02128",DESTASIO RHONDA,DESTASIO RHONDA,02128
22,EAST BOSTON,10 LEVERETT AV #1-C,EAST BOSTON MA,2128.0,"10 LEVERETT AV #1-C, 02128",TRIMBOLI ROSANNA,TRIMBOLI ROSANNA,02128
23,EAST BOSTON,10 LEVERETT AV #2-C,EAST BOSTON MA,2128.0,"10 LEVERETT AV #2-C, 02128",FIORE CHRISTINE,FIORE CHRISTINE,02128
35,EAST BOSTON,82 GLADSTONE ST # 1,EAST BOSTON MA,2128.0,"82 GLADSTONE ST #1, 02128",BURNOR EMILY A,BURNOR EMILY A,02128
75,EAST BOSTON,83 WEBSTER ST #1,EAST BOSTON MA,2128.0,"83 WEBSTER ST #1, 02128",OCONNELL KAELYN M,OCONNELL KAELYN M,02128
83,EAST BOSTON,27 MAVERICK SQ #9,EAST BOSTON MA,2128.0,"25 31 MAVERICK SQ #9, 02128",GUIMARAES ORLANDO S,GUIMARAES ORLANDO S,02128
86,EAST BOSTON,113 SUMNER ST #55,EAST BOSTON MA,2128.0,"113 SUMNER ST #55, 02128",DRAYTON LISA A,DRAYTON LISA A,02128


In [ ]:
def remove_sign(x, city):
  xs = x.split(', ')
  if '#' in x:
    address_pieces = xs[0].split(' ')
    clean = ' '.join([address_piece for address_piece in address_pieces if '#' not in address_piece])
    return clean.lower() + ', ' + city.lower() + ', ' + xs[1]
  else:
    return xs[0].lower() + ', ' + city.lower() + ', ' + xs[1]

In [ ]:
# test on the temp dataframe first
temp['address_on_map'] = temp.apply(lambda row: remove_sign(row['FULL_ADDRESS'], row['CITY']), axis=1 )
temp.head(30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CITY,MAIL_ADDRESS,MAIL_CS,ZIPCODE,FULL_ADDRESS,ADDRESSEE,OWNER,MAIL_ZIPCODE,address_on_map
18,EAST BOSTON,22 SWAN AVENUE,EAST BOSTON MA,2128.0,"12 LEVERETT AV #9-C, 02128",LEVERETT REALTY TRUST,LEVERETT REALTY TRUST,02128,"12 leverett av, east boston, 02128"
19,EAST BOSTON,12 LEVERETT AV #10C,E BOSTON MA,2128.0,"12 LEVERETT AV #10-C, 02128",LYNCH KATHLEEN,LYNCH KATHLEEN,02128,"12 leverett av, east boston, 02128"
20,EAST BOSTON,12 LEVERETT AV #11-C,EAST BOSTON MA,2128.0,"12 LEVERETT AV #11-C, 02128",DONES KEVEN,DONES KEVEN,02128,"12 leverett av, east boston, 02128"
21,EAST BOSTON,12 LEVERETT AV #12-C,EAST BOSTON MA,2128.0,"12 LEVERETT AV #12-C, 02128",DESTASIO RHONDA,DESTASIO RHONDA,02128,"12 leverett av, east boston, 02128"
22,EAST BOSTON,10 LEVERETT AV #1-C,EAST BOSTON MA,2128.0,"10 LEVERETT AV #1-C, 02128",TRIMBOLI ROSANNA,TRIMBOLI ROSANNA,02128,"10 leverett av, east boston, 02128"
23,EAST BOSTON,10 LEVERETT AV #2-C,EAST BOSTON MA,2128.0,"10 LEVERETT AV #2-C, 02128",FIORE CHRISTINE,FIORE CHRISTINE,02128,"10 leverett av, east boston, 02128"
35,EAST BOSTON,82 GLADSTONE ST # 1,EAST BOSTON MA,2128.0,"82 GLADSTONE ST #1, 02128",BURNOR EMILY A,BURNOR EMILY A,02128,"82 gladstone st, east boston, 02128"
75,EAST BOSTON,83 WEBSTER ST #1,EAST BOSTON MA,2128.0,"83 WEBSTER ST #1, 02128",OCONNELL KAELYN M,OCONNELL KAELYN M,02128,"83 webster st, east boston, 02128"
83,EAST BOSTON,27 MAVERICK SQ #9,EAST BOSTON MA,2128.0,"25 31 MAVERICK SQ #9, 02128",GUIMARAES ORLANDO S,GUIMARAES ORLANDO S,02128,"25 31 maverick sq, east boston, 02128"
86,EAST BOSTON,113 SUMNER ST #55,EAST BOSTON MA,2128.0,"113 SUMNER ST #55, 02128",DRAYTON LISA A,DRAYTON LISA A,02128,"113 sumner st, east boston, 02128"


In [ ]:
#The test looks successful, apply on the whole df
df['address_on_map'] = df.apply(lambda row: remove_sign(row['FULL_ADDRESS'], row['CITY']), axis=1 )
df.head(50)

,CITY,MAIL_ADDRESS,MAIL_CS,ZIPCODE,FULL_ADDRESS,ADDRESSEE,OWNER,MAIL_ZIPCODE,address_on_map
0,EAST BOSTON,288 LEXINGTON ST,EAST BOSTON MA,2128.0,"288 LEXINGTON ST, 02128",VIPOND ALEX RICHARD,VIPOND ALEX RICHARD,02128,"288 lexington st, east boston, 02128"
1,EAST BOSTON,335 E EAGLE ST,EAST BOSTON MA,2128.0,"335 E EAGLE ST, 02128",ALFARO MAREL W,ALFARO MAREL W,02128,"335 e eagle st, east boston, 02128"
2,EAST BOSTON,TWO JERICHO PZ WING C STE 11,JERICHO NY,2128.0,"BREMEN ST, 02128",GAVEGNANO HENRY J,GAVEGNANO HENRY J,11753,"bremen st, east boston, 02128"
3,EAST BOSTON,1 HARBORSIDE DR #200S,EAST BOSTON MA,2128.0,"5 VIENNA ST, 02128",MASSACHUSETTS PORT AUTHORITY,MASSACHUSETTS PORT AUTHORITY,02128,"5 vienna st, east boston, 02128"
4,EAST BOSTON,CHELSEA,EAST BOSTON MA,2128.0,"CHELSEA ST, 02128",MASS WATER RESOURCE AUTHORTY,MASS WATER RESOURCE AUTHORTY,02128,"chelsea st, east boston, 02128"
5,EAST BOSTON,581 BOYLSTON ST STE 604,BOSTON MA,2128.0,"546 SARATOGA ST, 02128",FIVE46 SARATOGA STREET LLC,FIVE46 SARATOGA STREET LLC,02116,"546 saratoga st, east boston, 02128"
6,EAST BOSTON,668 BENNINGTON ST,E BOSTON MA,2128.0,"668 BENNINGTON ST, 02128",TETZAGUIC HILDA,TETZAGUIC HILDA,02128,"668 bennington st, east boston, 02128"
7,EAST BOSTON,65 HORACE ST,EAST BOSTON MA,2128.0,"65 HORACE ST, 02128",COUGHLIN JAMES G,COUGHLIN JAMES G,02128,"65 horace st, east boston, 02128"
8,EAST BOSTON,37 ST ANDREW RD,EAST BOSTON MA,2128.0,"37 39 ST ANDREW RD, 02128",FERULLO JAMES A,FERULLO JAMES A,02128,"37 39 st andrew rd, east boston, 02128"
9,EAST BOSTON,1084 SARATOGA,EAST BOSTON MA,2128.0,"1084 SARATOGA ST, 02128",DA SILVA EDIVALDO BENTO,DA SILVA EDIVALDO BENTO,02128,"1084 saratoga st, east boston, 02128"


In [ ]:
# inspect
print(df.shape)

(170860, 9)


#A simple test: you can see how many parcels are there in the dataset belong to certain road or street.

In [ ]:
all_addresses = df['address_on_map'].to_list()
for a in all_addresses:
  if 'newburg' in a:
    print(a) 

15   newburg st, roslindale, 02131
151   newburg st, roslindale, 02131
155   newburg st, roslindale, 02131
159   newburg st, roslindale, 02131
163   newburg st, roslindale, 02131
167   newburg st, roslindale, 02131
171   newburg st, roslindale, 02131
168   newburg st, roslindale, 02131
164   newburg st, roslindale, 02131
160   newburg st, roslindale, 02131
156   newburg st, roslindale, 02131
144   newburg st, roslindale, 02131
140   newburg st, roslindale, 02131
138   newburg st, roslindale, 02131
134   newburg st, roslindale, 02131
130   newburg st, roslindale, 02131
126   newburg st, roslindale, 02131
122   newburg st, roslindale, 02131
118   newburg st, roslindale, 02131
newburg st, roslindale, 02131
110   newburg st, roslindale, 02131
60   newburg st, roslindale, 02131
64   newburg st, roslindale, 02131
106   newburg st, roslindale, 02131
100   newburg st, roslindale, 02131
96   newburg st, roslindale, 02131
92   newburg st, roslindale, 02131
newburg st, roslindale, 02131
82   newb